<a href="https://colab.research.google.com/github/vcautela/Estatistica/blob/main/Exercicio_Teste_de_Hip%C3%B3tese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=green size=10>CURSO DE ESTATÍSTICA - PARTE 3</font>

***

## Trabalho sobre testes de hipóteses

Utilizando os conhecimentos adquiridos em nosso treinamento realize o teste de hipóteses proposto abaixo.

Siga o roteiro proposto e vá completando as células vazias. Procure pensar em mais testes interessantes que podem ser realizados com o nosso dataset.

# <font color=green>DATASET DO PROJETO</font>
***

### Pesquisa Nacional por Amostra de Domicílios - 2015

A <b>Pesquisa Nacional por Amostra de Domicílios - PNAD</b> investiga anualmente, de forma permanente, características gerais da população, de educação, trabalho, rendimento e habitação e outras, com periodicidade variável, de acordo com as necessidades de informação para o país, como as características sobre migração, fecundidade, nupcialidade, saúde, segurança alimentar, entre outros temas. O levantamento dessas estatísticas constitui, ao longo dos 49 anos de realização da pesquisa, um importante instrumento para formulação, validação e avaliação de políticas orientadas para o desenvolvimento socioeconômico e a melhoria das condições de vida no Brasil.

### Fonte dos Dados

https://ww2.ibge.gov.br/home/estatistica/populacao/trabalhoerendimento/pnad2015/microdados.shtm

### Variáveis utilizadas

> ### Renda
> ***

Rendimento mensal do trabalho principal para pessoas de 10 anos ou mais de idade.

> ### Idade
> ***

Idade do morador na data de referência em anos.

> ### Altura (elaboração própria)
> ***

Altura do morador em metros.

> ### UF
> ***

|Código|Descrição|
|---|---|
|11|Rondônia|
|12|Acre|
|13|Amazonas|
|14|Roraima|
|15|Pará|
|16|Amapá|
|17|Tocantins|
|21|Maranhão|
|22|Piauí|
|23|Ceará|
|24|Rio Grande do Norte|
|25|Paraíba|
|26|Pernambuco|
|27|Alagoas|
|28|Sergipe|
|29|Bahia|
|31|Minas Gerais|
|32|Espírito Santo|
|33|Rio de Janeiro|
|35|São Paulo|
|41|Paraná|
|42|Santa Catarina|
|43|Rio Grande do Sul|
|50|Mato Grosso do Sul|
|51|Mato Grosso|
|52|Goiás|
|53|Distrito Federal|

> ### Sexo
> ***

|Código|Descrição|
|---|---|
|0|Masculino|
|1|Feminino|

> ### Anos de Estudo
> ***

|Código|Descrição|
|---|---|
|1|Sem instrução e menos de 1 ano|
|2|1 ano|
|3|2 anos|
|4|3 anos|
|5|4 anos|
|6|5 anos|
|7|6 anos|
|8|7 anos|
|9|8 anos|
|10|9 anos|
|11|10 anos|
|12|11 anos|
|13|12 anos|
|14|13 anos|
|15|14 anos|
|16|15 anos ou mais|
|17|Não determinados|
||Não aplicável|

> ### Cor
> ***

|Código|Descrição|
|---|---|
|0|Indígena|
|2|Branca|
|4|Preta|
|6|Amarela|
|8|Parda|
|9|Sem declaração|

#### <font color='red'>Observação</font>
***
> Os seguintes tratamentos foram realizados nos dados originais:
> 1. Foram eliminados os registros onde a <b>Renda</b> era inválida (999 999 999 999);
> 2. Foram eliminados os registros onde a <b>Renda</b> era missing;
> 3. Foram considerados somente os registros das <b>Pessoas de Referência</b> de cada domicílio (responsável pelo domicílio).

***
***

### Utilize a célula abaixo para importar as biblioteca que precisar para executar as tarefas
#### <font color='red'>Sugestões: pandas, numpy, scipy, statsmodels</font>

In [39]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from statsmodels.stats.weightstats import DescrStatsW, CompareMeans


### Importe o dataset e armazene o conteúdo em uma DataFrame

In [40]:
df = pd.read_csv('/content/dados.csv')

### Visualize o conteúdo do DataFrame

In [41]:
df.head()

,UF,Sexo,Idade,Cor,Anos de Estudo,Renda,Altura
0,11,0,23,8,12,800,1.603808
1,11,1,23,2,12,1150,1.739790
2,11,1,35,8,15,880,1.760444
3,11,0,46,2,6,3500,1.783158
4,11,1,47,8,9,150,1.690631


## <font color='red'>Problema</font>

Você é um pesquisador que estuda o mercado de trabalho brasileiro e resolve estudar as diferenças salariais dos trabalhadores dos estados do Rio de Janeiro e de São Paulo. Durante sua pesquisa você verifica que, aparentemente, os rendimentos dos trabalhadores no estado do Rio de Janeiro são mais baixos que os rendimentos dos trabalhadores no estado de São Paulo. Para confirmar esta conclusão realize um teste de hipótese de comparação de médias em cima de duas amostras de trabalhadores dos dois estados. Siga as seguintes etapas:

- Selecione duas amostras de **500 trabalhadores** para cada um dos dois estados (variável UF) para realizar o teste. Utilize o **parâmetro random_state=101**.
- Considere o **nível de significância de 5%**.
- Teste a hipótese de que a **renda média dos trabalhadores do Rio de Janeiro é menor que a renda média dos trabalhadores de São Paulo**.

---

### Seleção das amostras

In [42]:
RJ = df[['UF', 'Renda']].query('UF == 33').sample(n=500, random_state=101)
RJ = RJ.set_index('UF')
RJ.head()

,Renda
UF,
33,1300
33,35000
33,1300
33,2364
33,1500


In [43]:
SP = df[['UF', 'Renda']].query('UF == 35').sample(n=500, random_state=101)
SP = SP.set_index('UF')
SP.head()

,Renda
UF,
35,1800
35,700
35,8000
35,1800
35,2364


### Dados do problema
Obter média e desvio-padrão para as duas amostras

In [44]:
media_amostra_RJ = RJ.mean()
media_amostra_RJ[0]

2240.518

In [45]:
desvio_padrao_amostra_RJ = RJ.std()
desvio_padrao_amostra_RJ[0]

4643.83616186796

In [46]:
media_amostra_SP = SP.mean()
media_amostra_SP[0]

2839.352

In [47]:
desvio_padrao_amostra_SP = SP.std()
desvio_padrao_amostra_SP[0]

3699.384225434117

## Lembre-se...

<img src='https://caelum-online-public.s3.amazonaws.com/1229-estatistica-parte3/01/img014.png' width=90%>

---

### **Passo 1** - formulação das hipóteses $H_0$ e $H_1$

#### <font color='red'>Lembre-se, a hipótese nula sempre contém a alegação de igualdade</font>

 $ \mu_1= \$ média de renda do RJ

 $ \mu_2= \$ média de renda de SP

 $H_0: \mu_1 \geq \mu_2$

 $H_1: \mu_1 < \mu_2$

 ou

  $H_0: \mu_1 - \mu_2\geq $ 0

 $H_1: \mu_1 - \mu_2< $ 0

 Logo, teste unicaudal a esquerda


---

### **Passo 2** - escolha da distribuição amostral adequada

### O tamanho da amostra é maior que 30?
#### Resp.: sim

### Podemos afirmar que a população se distribui aproximadamente como uma normal?
#### Resp.: Não

### O desvio padrão populacional é conhecido?
#### Resp.: não


---

### **Passo 3** - fixação da significância do teste ($\alpha$)

In [48]:
significancia = 0.05
confianca = 1 - significancia
n_RJ = 500
n_SP = 500
D_0 = 0

In [49]:
probabilidade = significancia
probabilidade

0.05

In [55]:
z_alpha = norm.ppf(probabilidade)
z_alpha.round(2)

-1.64

---

### **Passo 4** - cálculo da estatística-teste e verificação desse valor com as áreas de aceitação e rejeição do teste

In [53]:
numerador = (media_amostra_RJ - media_amostra_SP) - D_0
denominador = np.sqrt((desvio_padrao_amostra_RJ**2 / n_RJ) + (desvio_padrao_amostra_SP**2 /n_SP))
z = numerador / denominador
z[0]

-2.255318273608558

---

### **Passo 5** - Aceitação ou rejeição da hipótese nula

### <font color='red'>Critério do valor crítico</font>

In [56]:
z <= z_alpha

Renda    True
dtype: bool

### <font color='red'>Critério do valor $p$</font>

### Utilize DescrStatsW

In [57]:
teste_rj = DescrStatsW(RJ)

In [58]:
teste_sp = DescrStatsW(SP)

In [59]:
test_A = teste_rj.get_compare(teste_sp)

In [61]:
z, p_valor = test_A.ztest_ind(alternative='smaller', value=D_0)
print('Estatística z -->', z)
print('p_valor -->', p_valor)

Estatística z --> [-2.25531827]
p_valor --> [0.01205668]


### Utilize CompareMeans

In [63]:
test_B = CompareMeans(teste_rj, teste_sp)

In [64]:
z, p_valor = test_B.ztest_ind(alternative='smaller', value=0)
print('Estatística z -->', z)
print('p_valor -->', p_valor)

Estatística z --> [-2.25531827]
p_valor --> [0.01205668]


In [65]:
p_valor <= significancia

array([ True])

### <font color='green'>Conclusão: (Digite sua conclusão aqui)</font>